Import libraries

In [2]:
#libraries to manipulate dataset
import pandas as pd 
import numpy as np 

# library to visualize the data
import matplotlib.pyplot as plt


#libraries for normalization and split data 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

#libraries for model
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

#libraries for evaluation th model

from sklearn.metrics import classification_report, confusion_matrix


import itertools


## Data pre-processing and  Visulationa dataframe

load product data

In [5]:
products = pd.read_csv("/dbfs/FileStore/tables/products.csv")
print("shape:", products.shape)
products.head()

,product_id,family_id,subfamily_id,price
0,151926,396066037,335531561,25.95
1,213413,552529755,11509337,19.95
2,310130,679611953,533441312,12.95
3,373883,775013441,62518413,17.95
4,455200,998145072,490222156,29.95


In [6]:
products.price.mean()

Out[3]: 25.34484840530253

load sales data

In [8]:
sales = pd.read_csv('/dbfs/FileStore/tables/sales_stock.csv' ) 
print("shape:",sales.shape)
sales.head()

,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


create data frame for sales by day

In [10]:
sales_day=pd.DataFrame(sales.groupby(['date_number']).sum()['sales']).reset_index()
sales_day.head()

,date_number,sales
0,0,30609
1,1,29806
2,2,28408
3,3,30983
4,4,40190


In [11]:
x = sales_day['date_number']
y = sales_day['sales']


plt.figure(figsize=(10,8))
plt.plot(x,y)
#plt.ylabel('Sales', fontsize=14)
plt.xlabel('Days',fontsize=14)
plt.title('Total sales by day',fontsize=14)
plt.legend(loc='upper lift', fontsize=14)
# Annotate arrow
plt.annotate('trend B',
            xy=(15,64200 ), xycoords='data',
            xytext=(-70, 20), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc,angleA=0,armA=50,rad=10"))

plt.annotate('Cut off the data',
            xy=(71,36000 ), xycoords='data',
            xytext=(-80, 100), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc,angleA=0,armA=80,rad=30"))

plt.annotate('trend A',
            xy=(79,81000 ), xycoords='data',
            xytext=(-80, 0), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc,angleA=0,armA=50,rad=10"))
plt.annotate('trend C',
            xy=(32,48000 ), xycoords='data',
            xytext=(-70, 20), textcoords='offset points',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc,angleA=0,armA=50,rad=10"))

a=plt.show()
display(a)

since the goal is to predicte demand for next 3 weeks, the dataset split train(0-71) and test(72-91)

In [13]:
train_sales = sales.loc[sales['date_number'] <= 71].reset_index(drop=True)
test_sales = sales.loc[sales['date_number'] >= 72].reset_index(drop=True)

In [14]:
print(train_sales.shape)
train_sales.head()

,date_number,product_id,color_id,size_id,sales,stock
0,0,310130,78467031,7026791,1,93
1,0,310130,78467031,8015276,2,62
2,0,310130,78467031,452052101,1,18
3,0,310130,78467031,541898038,2,42
4,0,310130,78467031,558685710,3,113


In [15]:
print(test_sales.shape)
test_sales.head()

,date_number,product_id,color_id,size_id,sales,stock
0,72,151926,909958217,54666687,1,73
1,72,151926,909958217,111455099,0,15
2,72,151926,909958217,242364222,2,109
3,72,151926,909958217,513017386,1,46
4,72,151926,909958217,820445075,0,66


pre-process sales and products dataset to train the model

In [17]:
sales_df=pd.DataFrame(train_sales.groupby(['product_id']).sum()['sales']).reset_index()
sales_df.head()

,product_id,sales
0,151926,83
1,213413,550
2,310130,492
3,455200,22
4,571044,386


Merge sales data with products to include the price and sales.

In [19]:
product_sales=pd.merge(products, train_sales, left_on='product_id', right_on='product_id')
product_sales.head()

,product_id,family_id,subfamily_id,price,date_number,color_id,size_id,sales,stock
0,151926,396066037,335531561,25.95,59,909958217,111455099,0,17
1,151926,396066037,335531561,25.95,59,909958217,242364222,3,63
2,151926,396066037,335531561,25.95,59,909958217,513017386,1,33
3,151926,396066037,335531561,25.95,59,909958217,820445075,1,42
4,151926,396066037,335531561,25.95,60,909958217,54666687,2,72


In [20]:
stay_columns=['product_id','price'] 

product_sales_final=pd.DataFrame(product_sales.groupby(stay_columns).sum()['sales']).reset_index()

product_sales_final.head()

,product_id,price,sales
0,151926,25.95,83
1,213413,19.95,550
2,310130,12.95,492
3,455200,29.95,22
4,571044,15.95,386


Load positions data

In [22]:
positions = pd.read_csv('/dbfs/FileStore/tables/positions.csv')
print("shape:",positions.shape)
positions.head()

,date_number,product_id,category_id,position
0,0,4450020,4461548,17
1,0,42147334,4461548,4
2,0,81131830,4461548,35
3,0,84035833,4461548,38
4,0,125252584,4461548,39


In [23]:
print('There are {} product_id in dataset.'.format(len(products['product_id'].unique())))
print('But there are only {} have positions.'.format(len(positions['product_id'].unique())))

There are 15238 product_id in dataset.
But there are only 9136 have positions.

In [24]:
print('There are {} product_id in dataset.'.format(len(products['product_id'].unique())))
print('But there are only {} have sales.'.format(len(sales['product_id'].unique())))

There are 15238 product_id in dataset.
But there are only 9595 have sales.

split positions data by day to traind the model

In [26]:
train_positions = positions.loc[positions['date_number'] <= 71].reset_index(drop=True)
test_positions = positions.loc[positions['date_number'] >= 72].reset_index(drop=True)

pre-process the positions

high_rank: minimum (best) position of each product

In [29]:
#insert column high_rank 
train_positions.loc[:,'high_rank']=train_positions.loc[:,'position'] 
# select the min value of position for each product in each day
high_rank=pd.DataFrame(train_positions.groupby(['product_id']).min()['position'] ).reset_index()
train_positions=train_positions.drop('high_rank', axis=1)

low_rank: maximum (worst) position of each product

In [31]:
# insert column low_rank 
train_positions.loc[:,'low_rank']=train_positions.loc[:,'position'] 
#select the max value of position for each product in each day
low_rank=pd.DataFrame(train_positions.groupby(['product_id']).max()['position'] ).reset_index()
train_positions=train_positions.drop('low_rank', axis=1)

calculate the mean position for every product

In [33]:
train_positions.loc[:,'rank_mean']=train_positions.loc[:,'position'] 
rank_mean=pd.DataFrame(train_positions.groupby(['product_id']).mean()['position']).reset_index()
rank_mean=round(rank_mean)
train_positions=train_positions.drop('rank_mean', axis=1)

creating final data fram for positions

In [35]:
positions_df=rank_mean.copy().rename(columns={"position": "rank_mean"}) 
positions_df.loc[:,'high_rank']=high_rank['position'] 
positions_df.loc[:,'low_rank']=low_rank['position'] 
print("shape:",positions_df.shape)
positions_df.head()

,product_id,rank_mean,high_rank,low_rank
0,151926,42.0,1,121
1,213413,71.0,4,255
2,310130,25.0,2,80
3,455200,28.0,1,137
4,571044,88.0,5,205


creating final training data frame

In [37]:
df=pd.merge(product_sales_final, positions_df,  left_on=['product_id'], right_on=['product_id']) 
print(df.shape)
df.head()

,product_id,price,sales,rank_mean,high_rank,low_rank
0,151926,25.95,83,42.0,1,121
1,213413,19.95,550,71.0,4,255
2,310130,12.95,492,25.0,2,80
3,455200,29.95,22,28.0,1,137
4,571044,15.95,386,88.0,5,205


In [38]:
df.corr()

,product_id,price,sales,rank_mean,high_rank,low_rank
product_id,1.000000,-0.004423,0.007819,0.005713,0.007879,0.006618
price,-0.004423,1.000000,-0.120031,0.001335,0.046639,0.009474
sales,0.007819,-0.120031,1.000000,-0.036398,-0.083394,0.005986
rank_mean,0.005713,0.001335,-0.036398,1.000000,0.601101,0.883619
high_rank,0.007879,0.046639,-0.083394,0.601101,1.000000,0.309536
low_rank,0.006618,0.009474,0.005986,0.883619,0.309536,1.000000


Sales by price

In [40]:
x = df['price']
y = df['sales']

plt.figure(figsize=(10,8))
plt.scatter(x,y,c='#b3003b',label='Products')
plt.ylabel('Sales', fontsize=14)
plt.xlabel('Price',fontsize=14)
plt.title('sales by price',fontsize=14)
plt.legend(loc='upper lift', fontsize=14)
a=plt.show()
display(a)

Note: When the price is low, sales are high

Sales by mean product positions

In [43]:
x = df['rank_mean']
y = df['sales']

plt.figure(figsize=(10,8))
plt.scatter(x,y,c='#86b300',label='Products')
plt.ylabel('Sales', fontsize=14)
plt.xlabel('Mean Positions',fontsize=14)
plt.title('Sales by Mean Product Positions',fontsize=14)
plt.legend(loc='upper lift', fontsize=14)
a=plt.show()
display(a)

Note: When the mean is low( best), the sales are high

Sales by high rank

In [46]:
x = df['high_rank']
y = df['sales']

plt.figure(figsize=(10,8))
plt.scatter(x,y,c='m',label='Products')
plt.ylabel('Sales', fontsize=14)
plt.xlabel('High positions',fontsize=14)
plt.title('Sales by High Ranks',fontsize=14)
plt.legend(loc='upper lift', fontsize=14)
a=plt.show()
display(a)

Note: the sales are high when the product has high rank

Sales by low_rank

In [49]:
x = df['low_rank']
y = df['sales']

plt.figure(figsize=(10,8))
plt.scatter(x,y,c='c',label='Products')
plt.ylabel('Sales', fontsize=14)
plt.xlabel('Low positins',fontsize=14)
plt.title('Sales by Low Ranks',fontsize=14)
plt.legend(loc='upper lift', fontsize=14)
a=plt.show()
display(a)

Note: sales are less when product rated by low

converted mean to two bins

In [52]:
bins=np.linspace(df.rank_mean.min(), df.rank_mean.max(), 5)

#customer reviews 10 to 1.
group_names=['4-stars','3-stars','2-stars','1-stars']

df['rank_level'] = pd.cut(df['rank_mean'],bins, labels=group_names,include_lowest=True)

df.head()

,product_id,price,sales,rank_mean,high_rank,low_rank,product_status,rank_level
0,151926,25.95,83,42.0,1,121,deal,4-stars
1,213413,19.95,550,71.0,4,255,deal,4-stars
2,310130,12.95,492,25.0,2,80,deal,4-stars
3,455200,29.95,22,28.0,1,137,deal,4-stars
4,571044,15.95,386,88.0,5,205,deal,4-stars


In [53]:
x = df['rank_level']
y = df['sales']

plt.figure(figsize=(10,5))
plt.bar(x,y,color='#006680',label='Products',width=0.4)
plt.ylabel('Sales', fontsize=14)
plt.xlabel('Position level',fontsize=14)
plt.title('Sales by Position Level',fontsize=14)
plt.legend(loc='upper lift', fontsize=14)
a=plt.show()
display(a)

Note: sales incresed by high rate

In [55]:
df['rank_level'] = pd.cut(df['rank_mean'],bins=2, labels=False)

df.rank_level.count()

Out[60]: 8681

converted the mean to deal or denial for product status.

In [57]:
df['product_status'] = df['rank_level'].apply(lambda x: "denial" if (x>0)  else "deal")
df=df.drop(columns=('rank_level'))
df.head()

,product_id,price,sales,rank_mean,high_rank,low_rank,product_status
0,151926,25.95,83,42.0,1,121,deal
1,213413,19.95,550,71.0,4,255,deal
2,310130,12.95,492,25.0,2,80,deal
3,455200,29.95,22,28.0,1,137,deal
4,571044,15.95,386,88.0,5,205,deal


visulaze the resutl

In [59]:
x = df['product_status']
y = df['sales']

plt.figure(figsize=(10,8))
plt.bar(x,y,color='#b33c00',label='Products',width=0.3)
plt.ylabel('Sales', fontsize=14)
plt.xlabel('Product status',fontsize=14)
plt.title('Sales by product status',fontsize=14)
plt.legend(loc='upper lift', fontsize=14)
a=plt.show()
display(a)

pre-processing testset

In [61]:
sales71=pd.DataFrame(test_sales.groupby(['product_id']).sum()['sales']).reset_index()

In [62]:
sales71=pd.merge(products, sales71, left_on='product_id', right_on='product_id')

In [63]:
stay_columns=['product_id','price'] 
sales71=pd.DataFrame(sales71.groupby(stay_columns).sum()['sales']).reset_index()

In [64]:
test_positions.loc[:,'high_rank']=test_positions.loc[:,'position'] 
high_rank=pd.DataFrame(test_positions.groupby(['product_id']).min()['position'] ).reset_index()
test_positions=test_positions.drop('high_rank', axis=1)

In [65]:
test_positions.loc[:,'low_rank']=test_positions.loc[:,'position'] 
low_rank=pd.DataFrame(test_positions.groupby(['product_id']).max()['position'] ).reset_index()
test_positions=test_positions.drop('low_rank', axis=1)

In [66]:
test_positions.loc[:,'rank_mean']=test_positions.loc[:,'position'] 
rank_mean=pd.DataFrame(test_positions.groupby(['product_id']).mean()['position']).reset_index()
rank_mean=round(rank_mean)
test_positions=test_positions.drop('rank_mean', axis=1)

In [67]:
positions_test=rank_mean.copy().rename(columns={"position": "rank_mean"}) 
positions_test.loc[:,'high_rank']=high_rank['position'] 
positions_test.loc[:,'low_rank']=low_rank['position'] 

In [68]:
test_df=pd.merge(sales71, positions_test,  left_on=['product_id'], right_on=['product_id']) 

In [69]:
test_df['rank_level'] = pd.cut(test_df['rank_mean'],bins=2, labels=False)

In [70]:
test_df['product_status'] = test_df['rank_level'].apply(lambda x: "denial" if (x>0)  else "deal")
test_df=test_df.drop(columns=('rank_level'))

### Selecting the feature

In [72]:
X_train = df[['price','sales','high_rank','low_rank']]

X_train[0:5]

,price,sales,high_rank,low_rank
0,25.95,83,1,121
1,19.95,550,4,255
2,12.95,492,2,80
3,29.95,22,1,137
4,15.95,386,5,205


In [73]:
y_train = df['product_status'].values
y_train[0:5]

Out[44]: array(['deal', 'deal', 'deal', 'deal', 'deal'], dtype=object)

In [74]:
X_test = test_df[['price','sales','high_rank','low_rank']]

X_test[0:5]

,price,sales,high_rank,low_rank
0,25.95,84,5,77
1,19.95,207,58,158
2,12.95,86,46,49
3,29.95,2,17,53
4,15.95,205,33,338


In [75]:
y_test = test_df['product_status'].values
y_test[0:5]

Out[46]: array(['deal', 'deal', 'deal', 'deal', 'deal'], dtype=object)

### Normalize Data

In [77]:
X_train= preprocessing.StandardScaler().fit(X_train).transform(X_train)
X_train[0:5]

/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 return self.partial_fit(X, y)
/local_disk0/tmp/1586616730400-0/PythonShell.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 from __future__ import absolute_import
Out[47]: array([[ 3.01076754e-02, -3.87211372e-01, -5.23594690e-01,
 -4.97840144e-01],
 [-3.29894708e-01, 4.99419266e-01, -4.05177498e-01,
 2.95576131e-01],
 [-7.49897488e-01, 3.89302398e-01, -4.84122292e-01,
 -7.40601841e-01],
 [ 2.70109264e-01, -5.03023940e-01, -5.23594690e-01,
 -4.03103873e-01],
 [-5.69896297e-01, 1.88054331e-01, -3.65705100e-01,
 -4.74718099e-04]])

In [78]:
X_test= preprocessing.StandardScaler().fit(X_test).transform(X_test)
X_test[0:5]

/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 return self.partial_fit(X, y)
/local_disk0/tmp/1586616730400-0/PythonShell.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
 from __future__ import absolute_import
Out[48]: array([[ 0.04723904, -0.09163238, -0.6261658 , -0.58905458],
 [-0.32521146, 0.45273479, 0.20027964, -0.19181723],
 [-0.75973705, -0.08278088, 0.01315992, -0.7263712 ],
 [ 0.29553938, -0.45454382, -0.43904607, -0.70675454],
 [-0.5735118 , 0.44388329, -0.18955311, 0.69093246]])

## Model

In [80]:
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(C=0.01).fit(X_train,y_train)
LR_model

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 FutureWarning)
Out[49]: LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
 intercept_scaling=1, max_iter=100, multi_class='warn',
 n_jobs=None, penalty='l2', random_state=None, solver='warn',
 tol=0.0001, verbose=0, warm_start=False)

In [81]:
LR_yhat = LR_model.predict(X_test)

In [82]:
LR_yhat

Out[51]: array(['deal', 'deal', 'deal', ..., 'deal', 'deal', 'deal'], dtype=object)

In [83]:
a = np.count_nonzero(LR_yhat == 'deal')
print('Total number of deal product is ',a)
#print(b = np.count_nonzero(LR_yhat == 'denial'))

Total number of deal product is 8020

In [84]:
b = np.count_nonzero(LR_yhat == 'denial')
print('Total number of denial product is ',b)

Total number of denial product is 413

## Model Evaluation

confusion_matrix

In [88]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test, LR_yhat, labels=['denial','deal']))

[[ 387 261]
 [ 26 7759]]

In [89]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, LR_yhat, labels=['denial','deal'])
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
a=plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['denial = 0','deal = 1'],normalize= True,  title='Confusion matrix')
display(a)